# Lab 2

#### Alan Abadzic, John Girard, Eric Laigaie, Garrett Shankel

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("NY_Listings_Validated.csv")

### Data Preparation Part 1

*Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.*

In [3]:
# Create Grade Variable and Encode it
def categorise(row):  
    if row['Review Scores Rating'] > 89:
        return 1
    else:
        return 0
    return 'IDK'

df['Grade'] = df.apply(lambda row: categorise(row), axis=1)

df['Grade'].value_counts(normalize=True)

# Filter to only useful columns
data = df[['Host Response Rate', 'Host Is Superhost', 'Host total listings count', 'City', 'Room type',
          'Accommodates', 'Bathrooms', 'Bedrooms', 'Price', 'Minimum nights', 'Maximum nights', 'Availability 365',
          'Number of reviews', 'Reviews per month', 'Grade']]

In [4]:
# One-hot Encode
city_one_hot = pd.get_dummies(data['City'])
room_one_hot = pd.get_dummies(data['Room type'])

data = data.drop('City',axis = 1)
data = data.drop('Room type',axis = 1)

data = data.join(city_one_hot)
data = data.join(room_one_hot)


# Map boolean to integer
data["Host Is Superhost"] = data["Host Is Superhost"].astype(int)

In [59]:
# Scale Data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

grade = data['Grade']
to_scale = data.drop("Grade", axis = 1)
cols = to_scale.columns

scaled = scaler.fit_transform(to_scale)

data = pd.DataFrame(scaled, columns = cols)
data['Grade'] = grade

In [8]:
data.head()

,Host Response Rate,Host Is Superhost,Host total listings count,Accommodates,Bathrooms,Bedrooms,Price,Minimum nights,Maximum nights,Availability 365,...,Reviews per month,Bronx,Brooklyn,Manhattan,Queens,Staten Island,Entire home/apt,Private room,Shared room,Grade
0,1.0,0.0,0.004086,0.000000,0.064516,0.000000,0.043043,0.000801,1.350418e-08,0.756164,...,0.072157,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,1.0,0.0,0.001021,0.000000,0.064516,0.000000,0.028028,0.000801,1.396984e-08,0.945205,...,0.062780,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,1.0,0.0,0.016343,0.200000,0.193548,0.076923,0.080080,0.001601,1.396984e-08,0.972603,...,0.156135,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
3,0.7,0.0,0.001021,0.200000,0.064516,0.000000,0.140140,0.000801,5.234033e-07,0.980822,...,0.027313,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,1.0,0.0,0.001021,0.066667,0.064516,0.000000,0.060060,0.000000,5.234033e-07,0.986301,...,0.150836,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1


### Data Preparation Part 2

*Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).*

Below is table that describes each variable in the dataset. This includes our two predictor variables, Price and Grade. Additionally, all variables are of type float64, except for the int64 Grade.

#### Variable Table
| Variable | Description |
| :-- | :-- |
| Host Response Rate | The rate that hosts respond to potential customers. |
| Host total listings count | The total number of airbnbs rentals the host owns. |
| Accommodates | The number of people this rental can accommodate. |
| Bathrooms | The number of bathrooms in this rental. |
| Bedrooms | The number of bedrooms in this rental. |
| Price | The base price for one night in this rental. |
| Minimum nights | The lowest amount of nights this rental can be booked for. |
| Maximum nights | The highest amount of nights this rental can be booked for. |
| Availability 365 | The proportion of nights in a year that the rental is available. |
| Reviews per month | The average number of reviews a rental receives in a month. |
| Host Is Superhost | One-hot variable: A "1" indicates the rental is operated by a Superhost. |
| Bronx | One-hot variable: A "1" indicates a Bronx-based rental. |
| Brooklyn | One-hot variable: A "1" indicates a Brooklyn-based rental. |
| Manhattan | One-hot variable: A "1" indicates a Manhattan-based rental. |
| Queens | One-hot variable: A "1" indicates a Queens-based rental. |
| Staten Island | One-hot variable: A "1" indicates a Staten Island-based rental. |
| Entire home/apt | One-hot variable: A "1" indicates this rental is an entire home or apartment. |
| Private room | One-hot variable: A "1" indicates this rental is a private room. |
| Shared room | One-hot variable: A "1" indicates this rental is a shared room. |
| Grade | One-hot variable: A "1" indicates this rental has a rating of .9 or more. |

### Modeling and Evaluation 1

Choose and explain your evaluation metrics that you will use (i.e., accuracy,
precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

#### Grade
OPTION A:
To measure a model's performance when predicting Grade classification, we will use an F1 score. We decided on this for a few reasons. First, there isn't a large difference in costs between a false negative and false positive. In the case of a false negative, customers will likely get a rental that exceeds their expectations. For a false positive, customers will have to deal with a worse rental than they might be expecting, but this rental could still has a passing review grade (70+). Additionally, the inbalanced class distribution (67% A's, 33% Non-A's) leads us to use F1 instead of plain accuracy due to F1's strength when dealing with unbalanced classes.

OPTION B:
To measure a model's performance when predicting Grade classification, we will use a sensitivity score. This is mainly due to the potential of a large negative costs for false positives. When a rental is given a false positive prediction, customers could be dealing with a rental that drastically fails their expectations.

#### Price
To measure a model's performance when predicting Price, we will use mean absolute percent error (MAPE). This option works well because it preserves the scale of the data (as opposed to squaring the difference), and provides a percentage value that is easier to interpret than a dollar value.

### Modeling and Evaluation 2

Choose the method you will use for dividing your data into training and
testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why
your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

#### Grade
On top of an 80% train, 20% test split, we will be using stratified 10-fold cross validation with grid search here. While non-stratified CV could also work, our imbalanced dataset (67% A's, 33% Non-A's) could yield folds of data that have very little Non-A's. Using stratification avoids this by holding these proportions constant across folds, leading to better training sets.

#### Price
On top of an 80% train, 20% test split, we will be using 10-fold cross validation with grid search here. We will not be using stratification because we're not dealing with inbalanced class labels that should be keep consistent. Since our response is continuous, we do not necessarily have to worry about certain outcomes not being selection for each fold.

In both cases, grid search helps us determine the best parameters.

### Modeling and Evaluation 3

Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

In [83]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time

# Split data into train and test
train, test = train_test_split(data, test_size=0.2)
train_y = train['Grade']
train_x = train.drop('Grade', axis=1)
test_y = train['Grade']
test_x = train.drop('Grade', axis=1)

#### Grade : Decision Tree

In [84]:
from sklearn.tree import DecisionTreeClassifier
# Make classifier
dt_clf = DecisionTreeClassifier()

# Parameter Grid
parameters = {'max_depth':(5, 10), 'min_samples_split':(2, 5, 10), 'splitter':('best', 'random')}

# Make GridSearch - inputting a cv value toggles stratification on
dt_cv = GridSearchCV(estimator = dt_clf, param_grid = parameters, cv = 10)

# Fit
start = time.time()
dt_cv.fit(train_x,train_y)
end = time.time()

# Find best parameters and predict on them
best_parameters = dt_cv.best_params_
preds = dt_cv.predict(test_x)
print("F1 Score for Random Forest: " + str(round(f1_score(test_y, preds)*100,2)) + "%")
print("Time Elapsed: " + str(round(end-start,2)))

F1 Score for Random Forest: 88.57%
Time Elapsed: 17.0


#### Grade : Logistic Regression

In [72]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter = 100000)

parameters = {'C':(1,10,100), 'solver':('lbfgs','saga')}
lr_cv = GridSearchCV(estimator = lr_clf, param_grid = parameters, cv = 10)
start = time.time()
lr_cv.fit(train_x,train_y)
end = time.time()

best_parameters = lr_cv.best_params_
preds = lr_cv.predict(test_x)
print("F1 Score for Logistic Regression: " + str(round(f1_score(test_y, preds)*100,2)) + "%")
print("Time Elapsed: " + str(round(end-start,2)))

F1 Score for Logistic Regression: 78.35
Time Elapsed: 200.06


#### Grade : KNN

In [74]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

parameters = {'n_neighbors':(3,5,7), 'algorithm':('auto','kd_tree')}
knn_cv = GridSearchCV(estimator = knn_clf, param_grid = parameters, cv = 10)
start = time.time()
knn_cv.fit(train_x,train_y)
end = time.time()

best_parameters = knn_cv.best_params_
preds = knn_cv.predict(test_x)
print("F1 Score for KNN: " + str(round(f1_score(test_y, preds)*100,2)) + "%")
print("Time Elapsed: " + str(round(end-start,2)))

F1 Score for KNN: 85.26
Time Elapsed: 357.3


In [90]:
# Reset Price column
data['Price'] = df['Price']

# Split data into train and test
train, test = train_test_split(data, test_size=0.2)
train_y = train['Price']
train_x = train.drop('Price', axis=1)
test_y = train['Price']
test_x = train.drop('Price', axis=1)

#### Price : Linear Regression

In [134]:
from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_absolute_error
linr_clf = LinearRegression()

parameters = {'fit_intercept':(True,False)}
linr_cv = GridSearchCV(estimator = linr_clf, param_grid = parameters, cv = 10)
start = time.time()
linr_cv.fit(train_x,train_y)
end = time.time()

best_parameters = linr_cv.best_params_
preds = linr_cv.predict(test_x)
check_MAPE = pd.DataFrame()
check_MAPE['real'], check_MAPE['preds'] = test_y, preds
check_MAPE['diff'] = abs(check_MAPE['preds'] - check_MAPE['real']) / check_MAPE['real']
check_MAPE = check_MAPE.replace(np.inf, np.nan)
print("Mean Absolute Percent Error for Linear Regression: " + str(round((check_MAPE['diff'].mean())*100,2)) + '%')
print("Time Elapsed: " + str(round(end-start,2)))

Mean Absolute Percent Error for Linear Regression: 39.3%
Time Elapsed: 1.38


#### Price : Elastic Net Regression

In [135]:
from sklearn.linear_model import ElasticNet

el_clf = ElasticNet()

parameters = {'l1_ratio':(.25,.5,.75),'alpha':(.1,1,10),'selection':('random','cyclic')}
el_cv = GridSearchCV(estimator = el_clf, param_grid = parameters, cv = 10)
start = time.time()
el_cv.fit(train_x,train_y)
end = time.time()

best_parameters = el_cv.best_params_
preds = el_cv.predict(test_x)
check_MAPE = pd.DataFrame()
check_MAPE['real'], check_MAPE['preds'] = test_y, preds
check_MAPE['diff'] = abs(check_MAPE['preds'] - check_MAPE['real']) / check_MAPE['real']
check_MAPE = check_MAPE.replace(np.inf, np.nan)
print("Mean Absolute Percent Error for Elastic Net Regression: " + str(round((check_MAPE['diff'].mean())*100,2)) + '%')
print("Time Elapsed: " + str(round(end-start,2)))

Mean Absolute Percent Error for Elastic Net Regression: 41.29%
Time Elapsed: 13.26


#### Price : Support Vector Regression

In [136]:
from sklearn.svm import SVR

svr_clf = SVR()

parameters = {'kernel':('linear','rbf')}#,'gamma':('scale','auto')}
svr_cv = GridSearchCV(estimator = svr_clf, param_grid = parameters, cv = 10)
start = time.time()
svr_cv.fit(train_x,train_y)
end = time.time()

best_parameters = svr_cv.best_params_
preds = svr_cv.predict(test_x)
check_MAPE = pd.DataFrame()
check_MAPE['real'], check_MAPE['preds'] = test_y, preds
check_MAPE['diff'] = abs(check_MAPE['preds'] - check_MAPE['real']) / check_MAPE['real']
check_MAPE = check_MAPE.replace(np.inf, np.nan)
print("Mean Absolute Percent Error for SVR: " + str(round((1 - check_MAPE['diff'].mean())*100,2)) + '%')
print("Time Elapsed: " + str(round(end-start,2)))

# THIS TAKES WAY TOO LONG - SHOULD SWITCH TO SOMETHING ELSE

KeyboardInterrupt: 

### Modeling and Evaluation 4

Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

SHOW MODEL ERROR/ACCURACY ON CHART WITH CONFIDENCE INTERVAL BARS? THAT WAY YOU COULD COMPARE

### Modeling and Evaluation 5

Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

NOT SURE WHAT THEY'RE TALKING ABOUT WITH STATISTICAL TESTS TO COMPARE PERFORMANCE, I'LL HAVE TO GO LOOK THAT UP. BUT WE CAN SPEAK TO ADVANTAGES LIKE INTERPRETABILITY / TIME / ACCURACY

### Modeling and Evaluation 6

Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

WE DID SOMETHING VERY SIMILAR IN THE MINILAB. JUST NEED TO DO IT FOR THE BEST PRICE AND GRADE MODEL

### Deployment

How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

GRADE: POSSIBLY NOT THE MOST USEFUL MODEL WITHOUT EXTRA DATA. YOU COULD TRY AND CLASSIFY RENTALS WITH SMALL NUMBER OF REVIEWS (BASICALLY, TRYING TO JUMP THE GUN AND FIND GRADE STATUS BEFORE GETTING ENOUGH REVIEWS), BUT THAT COULD BE EXTRAPOLATION.

PRICE: COULD BE USEFUL FOR AIRBNB TO FIND OVER/UNDERVALUED RENTALS TO CALIBRATE PRICING AND INCREASE BOOKINGS.

### Exceptional Work

You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?